In [0]:
!curl https://colab.chainer.org/install | sh -

!apt-get install graphviz
!pip install 'chaineripy'
!pip install 'chainercv'

import sys
import json

import chainer
from chainercv.datasets.voc.voc_bbox_dataset import VOCBboxDataset
from chainer.datasets import TransformDataset, ConcatenatedDataset
from chainercv.links import SSD300
from chainercv.extensions import DetectionVOCEvaluator
from chainercv.datasets import voc_bbox_label_names
from chainer import iterators

from google.colab import drive
import numpy as np


base_dir = './drive/My Drive/obj_detection'
sys.path.append(base_dir)
drive.mount('/content/drive')

from execute import run_train
from transform import Transform
from classifier import SSDClassifier

params = json.load(open(base_dir + '/params.json', "r"))

In [0]:
# Model
model = SSD300(n_fg_class=20, pretrained_model='imagenet')
model.use_preset('evaluate')
net = SSDClassifier(model, alpha=1, k=3)
chainer.cuda.get_device_from_id(0).use()
model.to_gpu()

# Data Augmentation
train = TransformDataset(
    ConcatenatedDataset(
            VOCBboxDataset(year='2007', split='trainval'),
            VOCBboxDataset(year='2012', split='trainval')
    ),
    Transform(model.coder, model.insize, model.mean)
)

test = VOCBboxDataset(year='2007', split='test', use_difficult=True, return_difficult=True)
train_iter = iterators.SerialIterator(train, params['batchsize'])
test_iter = iterators.SerialIterator(test, params['batchsize'], repeat=False, shuffle=False)

# Evaluator
evaluator = DetectionVOCEvaluator(test_iter, net.predictor, use_07_metric=True, label_names=voc_bbox_label_names)

In [0]:
params['save_trainer_interval'] = 6
params['l2_lambda']     = 0.0005
params['result_dir']    = 'drive/My Drive/study/result'
params['print_report']  = ["epoch", "main/loss", "main/loss/loc", "main/loss/conf", "val/main/map", "lr", "elapsed_time"]
run_train(train_iter, net, evaluator, **params)